In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
image_dir  = '/content/drive/MyDrive/resize_images'

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping



def load_images_and_labels(image_dir):
    images = []
    labels = []
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpeg'):
            img_path = os.path.join(image_dir, filename)
            img = load_img(img_path, target_size=(100, 100))
            img_array = img_to_array(img) / 255.0
            images.append(img_array)

            label = filename.split('_')[0]
            labels.append(label)

    return np.array(images), np.array(labels)

images, labels = load_images_and_labels(image_dir)

label_map = {label: idx for idx, label in enumerate(set(labels))}
labels = np.array([label_map[label] for label in labels])
labels = to_categorical(labels)


X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
train_datagen = ImageDataGenerator(

    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=len(label_map), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, epochs=200, validation_data=val_generator)

model.save('/content/drive/MyDrive/models/my_cnn_model.h5')


Epoch 1/200
484/484 [==============================] - 51s 94ms/step - loss: 1.8599 - accuracy: 0.3287 - val_loss: 1.8044 - val_accuracy: 0.3414
Epoch 2/200
484/484 [==============================] - 48s 99ms/step - loss: 1.5168 - accuracy: 0.4573 - val_loss: 1.3872 - val_accuracy: 0.4974
Epoch 3/200
484/484 [==============================] - 47s 97ms/step - loss: 1.3627 - accuracy: 0.5212 - val_loss: 1.3172 - val_accuracy: 0.5300
Epoch 4/200
484/484 [==============================] - 48s 99ms/step - loss: 1.2623 - accuracy: 0.5527 - val_loss: 1.1016 - val_accuracy: 0.6213
Epoch 5/200
484/484 [==============================] - 46s 95ms/step - loss: 1.2011 - accuracy: 0.5751 - val_loss: 1.2600 - val_accuracy: 0.5698
Epoch 6/200
484/484 [==============================] - 47s 96ms/step - loss: 1.1511 - accuracy: 0.5930 - val_loss: 1.1810 - val_accuracy: 0.5926
Epoch 7/200
484/484 [==============================] - 49s 102ms/step - loss: 1.1046 - accuracy: 0.6096 - val_loss: 1.3193 - val_a

KeyboardInterrupt: 